**jumbo rolls are folded in**

**changeover between jumbos are folded in**

**Minimize knife changes folded in**

need to ..

* show when an order in the future has been fullfilled.

* look forward in schedule to optimize new/better layouts

* make layout table editable

* speed test algorithm

add button to dashboard, calculate total deckle time, check why some orders aren't fullfilled 

# Simple Genetic Algorithm to Optimize Deckle Loss

In [23]:
import sys
sys.path.append("../")

In [24]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import math
from random import shuffle, choice
import copy
from utils import *
from genetic import *
from IPython.display import clear_output
import time
import datetime
import re

In [25]:
df = pd.read_excel('../data/200721_ New Way SCH W3-W6 W14 07.20.20.xlsx',
                   sheet_name='Schedule')

# the input schedule appears as a data table on the dashboard
# this is used to select what to deckle
df_input_schedule = pd.read_excel('../data/200721_ New Way SCH W3-W6 W14 07.20.20.xlsx',
                   sheet_name='Schedule')
df_input_schedule.insert(1, 'Technology', df_input_schedule['Description'].apply(lambda x: parse_description(x, 'tech')))
df_input_schedule.insert(2, 'Color', df_input_schedule['Description'].apply(lambda x: x.split(';')[1] if type(x) == str else None))
df_input_schedule.insert(3, 'Width', df_input_schedule['Description'].apply(lambda x: parse_description(x, 'width')))
df_input_schedule = df_input_schedule[[col for col in df_input_schedule.columns if 'Unnamed' not in str(col)]]
temp = df_input_schedule[['Customer Name', 'Technology', 'Color', 'Width', 'CYCLE / BUCKET',
                          'Description', 'Total LM Order QTY',
                          'Scheduled Ship Date', 'Date order is complete']]

In [26]:
# temp = temp[temp['Scheduled Ship Date'].apply(lambda x: type(x) == int)]                          
temp_json = temp.to_json()

In [27]:
temp2 = pd.read_json(temp_json, 
                     convert_dates=['Scheduled Ship Date', 'Date order is complete'])

## to_datetime and to_json use default units ms and ns, respectively
temp2['Scheduled Ship Date'] = pd.to_datetime(temp2['Scheduled Ship Date'], 
                                              errors='ignore', unit='ms')

In [28]:
### not currently used, grabs indicated information
customer_names = [i for i in df['Customer Name'].unique() if type(i) == str]
technologies = list(set([parse_description(i, 'tech') for i in df['Description'] 
                         if (type(i) == str)]))
technologies = [i for i in technologies if i]
colors = list(set([i[1] for i in list(df['Description']
                .str.split(';')) if type(i) == list]))
cycles = list(df['CYCLE / BUCKET'].unique())

In [29]:
customer = 'P & G'
technology = 'SAM' # Technology -> 
color = 'WHITE'
cycle = 'CYCLE 2'

df_filtered = df.loc[df['Customer Name'] == customer]
df_filtered = df_filtered.loc[df_filtered['Description'].str.contains(technology)]
df_filtered = df_filtered.loc[df_filtered['Description'].str.contains(color)] #CYAN, TEAL
df_filtered = df_filtered.loc[df_filtered['CYCLE / BUCKET'] == cycle]
df_filtered.insert(0, 'Block', 1)
df_filtered = df_filtered.reset_index(drop=True)
df_filtered['Order Number'] = df_filtered.index + 1

df_filtered['Width'] = pd.DataFrame(list(pd.DataFrame(list(df_filtered['Description']
                                                  .str.split(';')))[0].str.split('UN0')))[1]

In [30]:
### Detects if there is a break in the rolls coming of the
### line

prev_index = df_filtered.index[0] - 1
block = 1
for index in df_filtered.index:
    if index != prev_index + 1:
        block += 1
    df_filtered.loc[index, 'Block'] = block
    prev_index = index

In [31]:

lm = list(df_filtered.groupby('Description')['Total LM Order QTY'].sum().values)
widths = list(df_filtered.groupby('Width')['Total LM Order QTY'].sum().index.values.astype(int))
doffs_in_jumbo = 6
B = 4160
L = doff_length = 17000 * doffs_in_jumbo # df['LM putup']

neckin = [4, 4, 5, 7, 7, 7] # 158 missing in cycle 1, 4 mm knife in
w = list(np.array(widths) + np.array(neckin)) # the values used in the actual computation
q = [math.ceil(x/L) for x in lm] # jumbos needed per width, rounded up
s = BinPackingExample(w, q) # material orders (list of widths, 1 width = 1 jumbo)

print('The important variables', end='\n\n')
print('widths: {} (mm)'.format(widths))
print('neck in: {} (mm)'.format(neckin))
print('L (m): {}'.format(L))
print('undeckled jumbos needed: {}'.format(q))

The important variables

widths: [154, 158, 170, 208, 218, 234] (mm)
neck in: [4, 4, 5, 7, 7, 7] (mm)
L (m): 102000
undeckled jumbos needed: [96, 32, 318, 95, 158, 461]


In [32]:
# customer = 'P & G'
# product = 'SAM' # Technology -> 
# color = 'WHITE'
# cycle = 'CYCLE 2'

# df = pd.read_excel('../data/berry/200721_ New Way SCH W3-W6 W14 07.20.20.xlsx',
#                    sheet_name='Schedule')
# df = df.loc[df['Customer Name'] == customer]
# df = df.loc[df['Description'].str.contains(product)]
# df = df.loc[df['Description'].str.contains(color)] #CYAN, TEAL
# df = df.loc[df['CYCLE / BUCKET'] == cycle]
# df = df.reset_index(drop=True)
# df['Width'] = pd.DataFrame(list(pd.DataFrame(list(df['Description']
#                                                   .str.split(';')))[0].str.split('UN0')))[1]
# lm = list(df.groupby('Description')['Total LM Order QTY'].sum().values)
# widths = list(df.groupby('Width')['Total LM Order QTY'].sum().index.values.astype(int))
# doffs_in_jumbo = 6
# B = 4160
# L = doff_length = 17000 * doffs_in_jumbo # df['LM putup']

# neckin = [4, 4, 5, 7, 7, 7] # 158 missing in cycle 1, 4 mm knife in
# w = list(np.array(widths) + np.array(neckin))
# q = [math.ceil(x/L) for x in lm]
# print("max doffs needed without stock cutting: {}".format(q))
# s = BinPackingExample(w, q)
# df_json = df.to_json()

if we gave each order its own entire row we would produce a lot of wasted material

# Run Deckle Optimizer

In [33]:
sol, loss = find_optimum(s, B, widths, neckin, max_unique_products=4, 
                         loss_target=2, max_doff_layouts=10)

gene time: 1.0, chromosome time: 1600084514.0, ffd time: 1.0
loss: 0.962, best loss: 0.962, steps: 5, gene count: 5, unique doffs: 10, max unique on doff: 3
                  Formula Doffs Loss
0  170x6 + 218x3 + 234x10    46   25
1                  208x19     4   75
2                  154x26     2   52
3                  154x24     1  368
4          154x20 + 158x6     1   28
5                  158x25     1  110
6          158x1 + 170x22     1  148
7          170x20 + 208x3     1   15
8          208x16 + 218x3     1   45
9          218x17 + 234x1     1   94


# Build Layout for Dashboard View

In [34]:
layout_summary(sol, widths, neckin, B)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,Doffs,Loss
Doffs,,,,,,,,,,,,,,,,,,,,,
1,154.0,154.0,154.0,154.0,154.0,154.0,154.0,154.0,154.0,154.0,...,154.0,154.0,154.0,154.0,154.0,154.0,NaN,NaN,1,368.0
2,154.0,154.0,154.0,154.0,154.0,154.0,154.0,154.0,154.0,154.0,...,154.0,154.0,154.0,154.0,154.0,154.0,154.0,154.0,2,52.0
1,158.0,158.0,158.0,158.0,158.0,158.0,154.0,154.0,154.0,154.0,...,154.0,154.0,154.0,154.0,154.0,154.0,154.0,154.0,1,28.0
1,158.0,158.0,158.0,158.0,158.0,158.0,158.0,158.0,158.0,158.0,...,158.0,158.0,158.0,158.0,158.0,158.0,158.0,NaN,1,110.0
1,170.0,170.0,170.0,170.0,170.0,170.0,170.0,170.0,170.0,170.0,...,170.0,170.0,170.0,170.0,158.0,NaN,NaN,NaN,1,148.0
1,208.0,208.0,208.0,170.0,170.0,170.0,170.0,170.0,170.0,170.0,...,170.0,170.0,170.0,170.0,170.0,NaN,NaN,NaN,1,15.0
4,208.0,208.0,208.0,208.0,208.0,208.0,208.0,208.0,208.0,208.0,...,208.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,75.0
1,218.0,218.0,218.0,208.0,208.0,208.0,208.0,208.0,208.0,208.0,...,208.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,45.0
1,234.0,218.0,218.0,218.0,218.0,218.0,218.0,218.0,218.0,218.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,94.0


# Run Master Schedule, Optimizing Late Orders

In [35]:
setup_df = pd.read_excel('../data/200725_WAVA Deckle Optimization Parameters Rev1.xlsx',
                         sheet_name='Deckle Parameters')
speed_df = pd.read_excel('../data/200725_WAVA Deckle Optimization Parameters Rev1.xlsx',
                         sheet_name='Product Parameters')

In [37]:
start_date = datetime.datetime(2020,8,1)
start_date

datetime.datetime(2020, 8, 1, 0, 0)

In [41]:
master = optimize_schedule(sol, widths, neckin, df_filtered, L, setup_df, 
                           speed_df, doffs_in_jumbo, start_date, 'asdf')
master.head(15)

,Formula,Doffs,Total LM Order QTY,Width,Scheduled Ship Date,Order Number,Completion Date
0,NaN,NaN,NaN,NaN,NaT,Changeover,2020-08-01 00:50:00.000000
1,170x6 + 218x3 + 234x10,6,2.13675e+06,234,2020-08-10,9,2020-08-01 02:08:27.692308
2,NaN,NaN,NaN,NaN,NaT,Changeover,2020-08-01 02:23:27.692308
3,170x6 + 218x3 + 234x10,6,2.13675e+06,234,2020-08-10,9,2020-08-01 03:41:55.384616
4,NaN,NaN,NaN,NaN,NaT,Changeover,2020-08-01 03:56:55.384616
5,170x6 + 218x3 + 234x10,6,2.13675e+06,234,2020-08-10,9,2020-08-01 05:15:23.076924
6,NaN,NaN,NaN,NaN,NaT,Changeover,2020-08-01 05:30:23.076924
7,170x6 + 218x3 + 234x10,6,2.13675e+06,234,2020-08-14,1 and 15,2020-08-01 06:48:50.769232
8,NaN,NaN,NaN,NaN,NaT,Changeover,2020-08-01 07:03:50.769232
9,170x6 + 218x3 + 234x10,6,2.13675e+06,234,2020-08-14,1 and 15,2020-08-01 08:22:18.461540


In [42]:
master.iloc[-1]['Completion Date'] - master.iloc[0]['Completion Date']

Timedelta('4 days 00:54:13.846172')

In [43]:
deckle_time = master.iloc[-1]['Completion Date'] - master.iloc[0]['Completion Date']

In [57]:
days = math.floor(deckle_time.total_seconds()/60/60/24)
hours = ((deckle_time.total_seconds()/60/60/24)%
 (math.floor(deckle_time.total_seconds()/60/60/24)))*24

0.9038461588888822

In [66]:
var = ("{}".format(deckle_time)).split(".")[0]
print(var)

4 days 00:54:13


In [49]:
print(deckle_time.strftime("%b %d %Y %H:%M:%S"))

AttributeError: 'Timedelta' object has no attribute 'strftime'